In [1]:
import pandas as pd
import numpy as np

In [2]:
# importing, changing pandas data to numpy array and shuffling it
iris = pd.read_csv("Iris.csv")
iris_np = iris.to_numpy()
np.random.seed(42)
np.random.shuffle(iris_np)

In [3]:
location_of_name = len(iris_np[0]) - 1 #where the iris names are located

In [4]:
#this was supposed to add more Complex variables like x**2 to make the model more flexible but when I run the calculations I
#get nan, this cell works and dose it job, but Im not using it
def ComplexVariables(X):
    X = np.array(X)
    temp = np.zeros((len(X), (len(X[0])*2)))
    for i in range(len(X)):
        for j in range(len(X[0])):
            temp[i][j] = X[i][j]
        for j in range(len(X[0])):
            temp[i][j + len(X[0])] = np.power(X[i][j], 2)
    return temp

In [5]:
#calculating the Cost with delta modifier to protect from overtraining
def Cost(X, theta, y, delta):
    prediction = sigmoid(np.dot(X, np.transpose(theta)))
    y = np.array(y)
    cost = sum( -(y * np.log(prediction)) - (1 - y) * np.log(1 - prediction))/len(y) + (sum(np.power(theta, 2))) * delta/(2*len(y))
    return cost

In [6]:
def sigmoid(prediction):
    prediction = list(-np.array(prediction))#this is to reverse the sign
    sigmoid = 1 / (1 + np.exp(prediction))
    return sigmoid

In [7]:
#calculating the Gradient with delta modifier to protect from overtraining
def Gradient(X, theta, y, alfa, delta):
    prediction = sigmoid(np.dot(X, np.transpose(theta)))
    temp_theta = theta.copy()
    temp_theta[0] = 0
    for i in range(len(theta)):
        theta[i] = theta[i] - (alfa * sum((prediction-y) * X[:,i]/len(y)) + (delta/len(y)) * temp_theta[i])
    return theta

In [8]:
def CostSum(training_set, theta, y_train, iris_names, delta):
    cost_sum = 0
    for i in range(len(iris_names)):
        classes = (y_train == i).astype(int)
        cost_sum += Cost(training_set, theta[i], classes, delta)
    print("the Error of the funcion is ", cost_sum)

In [9]:
def succes_rate(X, theta, y):
    predictions = list(np.dot(X, np.transpose(theta)))
    predictions = np.argmax(predictions, axis=1)
    succes_rate = (y == predictions).astype(int)
    succes_rate = sum(succes_rate)/len(y)
    print("succes rate", succes_rate)

In [10]:
# Changes "iris-name" to numbers from 0 to amount of different names
iris_names = []

for i in range(len(iris_np)):
    has_value = False
    for j in range(len(iris_names)):
        if(iris_names[j] == iris_np[i][location_of_name]):
            iris_np[i][location_of_name] = j
            has_value = True
    if has_value == False:
        iris_names.append(iris_np[i][location_of_name])
        iris_np[i][location_of_name] = len(iris_names) - 1
        has_value = False

In [11]:
# getting the answers and deleting unnecessary data
iris_y = iris_np[:,location_of_name]
iris_np = np.delete(iris_np, location_of_name, 1)
iris_np = np.delete(iris_np, 0, 1)
iris_np = np.append(np.ones((len(iris_np), 1)), iris_np, axis=1)

In [12]:
# splitting data into training_set 80% and testing_set 20%, setting delta(helps to avoid overfitting)
location_of_name = len(iris_np[0]) - 1
training_ratio = 0.2
delta = 0.1
#iris_np = ComplexVariables(iris_np)
training_set = iris_np[:int((1-training_ratio) * len(iris_np)),:].copy()
testing_set = iris_np[int((1-training_ratio) * len(iris_np)):,:].copy()
y_train = iris_y[:int((1-training_ratio) * len(iris_np))].copy()
y_test = iris_y[int((1-training_ratio) * len(iris_np)):].copy()

In [13]:
#generating theta
theta = np.random.rand(len(iris_names), len(training_set[0]))

In [14]:
#Cost before training
CostSum(training_set, theta, y_train, iris_names, delta)

the Error of the funcion is  15.72682452555025


In [15]:
#trains for 10000 iterations
for i in range(len(iris_names)):
    classes = (y_train == i).astype(int)
    for j in range(10000):
        Gradient(training_set, theta[i], classes, 0.1, delta)#0.1 is how big are the steps in gradient descent

In [16]:
#Cost after training
CostSum(training_set, theta, y_train, iris_names, delta)

the Error of the funcion is  0.6876999762494191


In [17]:
# succes rate of training set
succes_rate(training_set ,theta, y_train)

succes rate 0.9416666666666667


In [18]:
# succes rate of testing set
succes_rate(testing_set ,theta, y_test)

succes rate 1.0
